1. 使用 AWQ 量化 OPT-6.7B 模型.

In [1]:
from transformers import AutoTokenizer, AwqConfig
from awq import AutoAWQForCausalLM

model_path = "../../../opt-6.7b"
quant_path = "models/facebook/opt-6.7b-awq4"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}

tokenizer = AutoTokenizer.from_pretrained(model_path)

In [2]:
model = AutoAWQForCausalLM.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model.quantize(tokenizer, quant_config=quant_config)

quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config
print(quantization_config)

/home/sheng/python-env/learn-llm/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating validation split: 0 examples [00:00, ? examples/s]

AWQ: 100%|██████████| 32/32 [24:42<00:00, 46.33s/it]

{'quant_method': <QuantizationMethod.AWQ: 'awq'>, 'bits': 4, 'group_size': 128, 'zero_point': True, 'version': <AWQLinearVersion.GEMM: 'gemm'>, 'backend': <AwqBackendPackingMethod.AUTOAWQ: 'autoawq'>, 'fuse_max_seq_len': None, 'modules_to_fuse': None, 'do_fuse': False}


In [4]:
# 保存模型权重
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)  # 保存分词器

('models/facebook/opt-6.7b-awq4/tokenizer_config.json',
 'models/facebook/opt-6.7b-awq4/special_tokens_map.json',
 'models/facebook/opt-6.7b-awq4/vocab.json',
 'models/facebook/opt-6.7b-awq4/merges.txt',
 'models/facebook/opt-6.7b-awq4/added_tokens.json',
 'models/facebook/opt-6.7b-awq4/tokenizer.json')

In [5]:
model.eval()

OptAWQForCausalLM(
  (model): OPTForCausalLM(
    (model): OPTModel(
      (decoder): OPTDecoder(
        (embed_tokens): Embedding(50272, 4096, padding_idx=1)
        (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
        (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-31): 32 x OPTDecoderLayer(
            (self_attn): OPTAttention(
              (k_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=True, w_bit=4, group_size=128)
              (v_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=True, w_bit=4, group_size=128)
              (q_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=True, w_bit=4, group_size=128)
              (out_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=True, w_bit=4, group_size=128)
            )
            (activation_fn): ReLU()
            (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affin

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(quant_path)
model = AutoModelForCausalLM.from_pretrained(quant_path, device_map="cuda").to(0)

In [7]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to see you're still around.
I'm still around, just not as much as I used to be. I'm still here though.
